In [ ]:
#|hide
#|default_exp export

# nbprocess.export
- Exporting a notebook to a library

In [ ]:
#|export
from nbprocess.read import *
from nbprocess.maker import *
from nbprocess.imports import *

from fastcore.script import *
from fastcore.imports import *
from fastcore.xtras import *

from collections import defaultdict
from pprint import pformat
from inspect import signature,Parameter
import ast,contextlib,copy

In [ ]:
from fastcore.test import *
from pdb import set_trace
from importlib import reload
import shutil

## NBProcessor -

Special comments at the start of a cell can be used to provide information to `nbprocess` about how to process a cell, so we need to be able to find the location of these comments.

In [ ]:
minimal = read_nb('../tests/minimal.ipynb')

In [ ]:
#|export
def extract_comments(ss):
    "Take leading comments from lines of code in `ss`, remove `#|`, and split"
    ss = ss.splitlines()
    first_code = first(i for i,o in enumerate(ss) if not o.strip() or not re.match(r'\s*#\|', o))
    return L((s.strip()[2:]).strip().split() for s in ss[:first_code]).filter()

nbprocess comments start with `#`, followed by whitespace delimited tokens, which `extract_comments` extracts from the start of a cell, up until a blank line or a line containing something other than comments:

In [ ]:
exp  = """#|export module
#| hide
1+2
#bar"""
test_eq(extract_comments(exp), [['export', 'module'],['hide']])

In [ ]:
#|export
@functools.lru_cache(maxsize=None)
def _param_count(f):
    "Number of parameters accepted by function `f`"
    params = list(signature(f).parameters.values())
    # If there's a `*args` then `f` can take as many params as needed
    if first(params, lambda o: o.kind==Parameter.VAR_POSITIONAL): return 99
    return len([o for o in params if o.kind in (Parameter.POSITIONAL_ONLY,Parameter.POSITIONAL_OR_KEYWORD)])

In [ ]:
#|export
def opt_set(var, newval):
    "newval if newval else var"
    return newval if newval else var

In [ ]:
#|export
class NBProcessor:
    "Process cells and nbdev comments in a notebook"
    def __init__(self, path=None, procs=None, preprocs=None, postprocs=None, nb=None, debug=False):
        self.nb = read_nb(path) if nb is None else nb
        self.procs,self.preprocs,self.postprocs = map(L, (procs,preprocs,postprocs))
        self.debug = debug

    def _process_cell(self, cell):
        self.cell = cell
        if not hasattr(cell, '_comments'): cell._comments = extract_comments(cell.source)
        for proc in self.procs:
            if cell.cell_type=='code': cell._comments.map(self._process_comment,proc)
            if callable(proc): cell = opt_set(cell, proc(cell))

    def _process_comment(self, proc, comment):
        cmd,*args = comment
        f = getattr(proc, f'_{cmd}_', None)
        if not f or _param_count(f)-1<len(args): return True
        if self.debug: print(cmd, args, f)
        return f(self, *args)
        
    def process(self):
        "Process all cells with `process_cell`"
        for proc in self.preprocs: self.nb = opt_set(self.nb, proc(self.nb))
        for i in range_of(self.nb.cells): self._process_cell(self.nb.cells[i])
        self.nb.cells = [c for c in self.nb.cells if c and getattr(c,'source',None) is not None]
        for proc in self.postprocs: self.nb = opt_set(self.nb, proc(self.nb))

In [ ]:
class _PrintExample:
    def _printme_(self, nbp, to_print): print(to_print)

everything_fn = '../tests/01_everything.ipynb'
proc = NBProcessor(everything_fn, _PrintExample())
proc.process()

testing


## Notebook Processors

In [ ]:
_test_file = '../tests/docs_test.ipynb'

def _run_procs(procs=None, preprocs=None, postprocs=None):
    nbp = NBProcessor(_test_file, procs, preprocs=preprocs, postprocs=postprocs)
    nbp.process()
    return '\n'.join([cell.source for cell in nbp.nb.cells])

### Injecting Metadata Into Cells -

In [ ]:
#|export
_re_meta= r'^\s*#(?:cell_meta|meta):\S+\s*[\n\r]'

def inject_meta(cell):
    "Inject metadata into a cell for further preprocessing with a comment."
    _pattern = r'(^\s*#(?:cell_meta|meta):)(\S+)(\s*[\n\r])'
    if cell.cell_type == 'code' and re.search(_re_meta, cell.source, flags=re.MULTILINE):
        cell_meta = re.findall(_pattern, cell.source, re.MULTILINE)
        d = cell.metadata.get('nbprocess', {})
        for _, m, _ in cell_meta:
            if '=' in m:
                k,v = m.split('=')
                d[k] = v
            else: print(f"Warning cell_meta:{m} does not have '=' will be ignored.")
        cell.metadata['nbprocess'] = d

In [ ]:
#|export
def show_meta(cell):
    "Show cell metadata"
    meta = cell.metadata.get('nbprocess')
    if meta: print(meta)

To inject metadata make a comment in a cell with the following pattern: `#cell_meta:{key=value}`. Note that `#meta` is an alias for `#cell_meta`

For example, at the moment, this notebook has no cells with metadata, which we can see b using `show_meta`:

In [ ]:
_run_procs([show_meta]);

However, after we process this notebook with `inject_meta`, the appropriate metadata will be injected:

In [ ]:
_run_procs([inject_meta,show_meta]);

### Insert Warning Into Markdown -

In [ ]:
#| export
def insert_warning(nb):
    "Insert Autogenerated Warning Into Notebook after the first cell."
    content = "<!-- WARNING: THIS FILE WAS AUTOGENERATED! DO NOT EDIT! -->"
    nb.cells.insert(1, AttrDict(cell_type='markdown', metadata={}, source=content))

This preprocessor inserts a warning in the markdown destination that the file is autogenerated.  This warning is inserted in the second cell so we do not interfere with front matter.

In [ ]:
res = _run_procs(preprocs=[insert_warning])
assert "<!-- WARNING: THIS FILE WAS AUTOGENERATED!" in res

In [ ]:
 _run_procs([inject_meta, show_meta]);

### Remove cell bits based on tags

In [ ]:
#|export
def update_tags(cell):
    root = cell.metadata.get('nbprocess', {})
    tags = root.get('tags', root.get('tag')) # allow the singular also
    if tags: cell.metadata['tags'] = cell.metadata.get('tags', []) + tags.split(',')

`update_tags` is meant to be used with `inject_meta` to configure the visibility of cells in rendered docs.

In [ ]:
def TagRemove(cell_tag=None, outputs_tag=None, input_tag=None):
    def _inner(cell):
        tags = nested_idx(cell, 'metadata', 'tags') or []
        if cell_tag in tags: cell['source'] = None
        if outputs_tag in tags and 'outputs' in cell: del(cell['outputs'])
        if input_tag in tags: del(cell['source'])
    return _inner

In [ ]:
res = _run_procs([inject_meta, update_tags, TagRemove("remove_cell",'remove_output','remove_input')])
assert 'you will not be able to see this cell at all either' not in res

## Export -

In [ ]:
#|skip
basic_export_nb2('00_read.ipynb', 'read')
basic_export_nb2('01_maker.ipynb', 'maker')
basic_export_nb2('02_process.ipynb', 'process')

g = exec_new('import nbprocess.process')
assert hasattr(g['nbprocess'].process, 'NBProcessor')